### Import the Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import math

### Reading the contents of the file - "canopy_technical_test_output.html"
- This file is obtained by using the pdftotext library to convert the pdf file into a html file. 
- By running the following command in command prompt: <br>
pdftotext -bbox-layout -htmlmeta "canopy_technical_test_input.pdf" "canopy_technical_test_output.html"


In [65]:
f = open("canopy_technical_test_output.html", "r")
content = f.read()
content

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<title></title>\n<meta name="Creator" content="Aspose Ltd."/>\n<meta name="Producer" content="Aspose.PDF for .NET 18.4"/>\n<meta name="CreationDate" content=""/>\n</head>\n<body>\n<doc>\n  <page width="841.890000" height="595.276000">\n    <flow>\n      <block xMin="110.551200" yMin="47.531705" xMax="247.063958" yMax="70.931835">\n        <line xMin="110.551200" yMin="47.531705" xMax="247.063958" yMax="70.931835">\n          <word xMin="110.551200" yMin="47.531705" xMax="169.033525" yMax="70.931835">Account</word>\n          <word xMin="173.533550" yMin="47.531705" xMax="247.063958" yMax="70.931835">Statement</word>\n        </line>\n      </block>\n      <block xMin="110.551200" yMin="74.332500" xMax="242.202200" yMax="91.232500">\n        <line xMin="110.551200" yMin="74.332500" xMax="242.202200" yMax="91.232500

### Using BeautifulSoup package to parse the html file.

In [66]:
soup = BeautifulSoup(content, "html.parser")
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title></title>
<meta content="Aspose Ltd." name="Creator"/>
<meta content="Aspose.PDF for .NET 18.4" name="Producer"/>
<meta content="" name="CreationDate"/>
</head>
<body>
<doc>
<page height="595.276000" width="841.890000">
<flow>
<block xmax="247.063958" xmin="110.551200" ymax="70.931835" ymin="47.531705">
<line xmax="247.063958" xmin="110.551200" ymax="70.931835" ymin="47.531705">
<word xmax="169.033525" xmin="110.551200" ymax="70.931835" ymin="47.531705">Account</word>
<word xmax="247.063958" xmin="173.533550" ymax="70.931835" ymin="47.531705">Statement</word>
</line>
</block>
<block xmax="242.202200" xmin="110.551200" ymax="91.232500" ymin="74.332500">
<line xmax="242.202200" xmin="110.551200" ymax="91.232500" ymin="74.332500">
<word xmax="171.313200" xmin="110.551200" ymax="91.232500" ymin="74.332500">31.03.2

#### For Parsing the Table correctly, I will be reading the file line by line - i.e. by parsing all the <lines> in the html file, as shown below:

In [67]:
lines = soup.find_all("line")
lines

[<line xmax="247.063958" xmin="110.551200" ymax="70.931835" ymin="47.531705">
 <word xmax="169.033525" xmin="110.551200" ymax="70.931835" ymin="47.531705">Account</word>
 <word xmax="247.063958" xmin="173.533550" ymax="70.931835" ymin="47.531705">Statement</word>
 </line>,
 <line xmax="242.202200" xmin="110.551200" ymax="91.232500" ymin="74.332500">
 <word xmax="171.313200" xmin="110.551200" ymax="91.232500" ymin="74.332500">31.03.2018</word>
 <word xmax="178.190200" xmin="174.563200" ymax="91.232500" ymin="74.332500">-</word>
 <word xmax="242.202200" xmin="181.440200" ymax="91.232500" ymin="74.332500">30.04.2018</word>
 </line>,
 <line xmax="273.311201" xmin="110.551200" ymax="109.667400" ymin="96.667388">
 <word xmax="143.681200" xmin="110.551200" ymax="109.667400" ymin="96.667400">Account</word>
 <word xmax="247.891201" xmin="146.181200" ymax="109.667400" ymin="96.667400">SG1234567-01-01-JPY01</word>
 <word xmax="257.841201" xmin="250.391201" ymax="109.667388" ymin="96.667388">in</w

### The following steps are followed to get all the y_positions and column_name details:
1. get_line_positions function to get the x and y positions of all the lines.
2. x_positions that contain only the xmin and xmax values.
3. y_positions that contain only the ymin and ymax values.
4. Sorting the y_positions by ymin values so that we will be able to read the contents from top to bottom.
5. get_columns function to get the x and y positions associated with the column_names.
    - For every y-position, I am iterating through every line tag to get the text, xmin, xmax, ymin and ymax associated with it.
    - If there are multiple texts associated with the same <line> tag, then I am concatenating them into a single text.
    - The way I am identifying columns is : When there is more than 1 <line> object in a line (i.e as we go down the list of y_position).
6. These values will be used while parsing the html file for table contents.
7. get_column_names - The funcion get_columns also includes the column names. This function retieves just the column_names from the output of the previous function.



In [68]:
### Function to Get the xmin, xmax, ymin, ymax values of every line in lines: i.e. To get the x and y postitions of lines:
def get_line_positions(lines):
    df_lines_positions = pd.DataFrame()
    for line in lines:
        row = []
        x_min = float(line.get('xmin'))
        x_max = float(line.get('xmax'))
        y_min = float(line.get('ymin'))
        y_max = float(line.get('ymax'))
        row = [[x_min, x_max, y_min, y_max]]
        df_lines_positions = df_lines_positions.append(row)
    df_lines_positions.columns = ["xmin", "xmax","ymin","ymax"]
    return df_lines_positions

In [69]:
df_lines_positions = get_line_positions(lines)

In [70]:
## Get the X positions and y positions seperately from the lines_positions:
x_positions = df_lines_positions[["xmin","xmax"]]
x_positions = x_positions.drop_duplicates()
y_positions = df_lines_positions[["ymin","ymax"]]
y_positions = y_positions.drop_duplicates()

In [71]:
## Sort the y positions so that we will be able to read the file from top to bottom using the values in y_positions.
y_positions = y_positions.sort_values("ymin")

In [72]:
### Function to parse the file to get the column positions for the table:
def get_columns(y_positions, lines):
    for y_pos in y_positions.iterrows():
        line_list = []
        for line in lines:
            if math.floor(float(line.get('ymin'))) == math.floor(y_pos[1]["ymin"]):
                s = line.get_text()
                s = ''.join(s)
                s = s.replace("\n", " ")
                s = (s, float(line.get('xmin')), float(line.get('xmax')), float(line.get('ymin')), float(line.get('ymax'))) 
                
                y = (float(line.get('ymin')), float(line.get('ymax')))
                line_list.append(s)
        line_list = sorted(line_list, key=lambda x: x[1])
        if (len(line_list) > 1):
            columns_positions = line_list
            return columns_positions, y

In [73]:
columns_positions, y_col_positions = get_columns(y_positions, lines)
print(columns_positions)
print(y_col_positions)

[(' Booking Date ', 113.6458, 166.2458, 140.6675, 153.6675), (' Txn Date ', 198.3006, 233.0006, 140.6675, 153.6675), (' Booking Text ', 257.7085, 308.1885, 140.6675, 153.6675), (' Value Date ', 536.1123, 578.8823, 140.6675, 153.6675), (' Debit ', 643.3031, 664.2631, 140.6675, 153.6675), (' Credit ', 720.917, 744.797, 140.6675, 153.6675), (' Balance ', 793.399, 824.439, 140.6675, 153.6675)]
(140.6675, 153.6675)


In [74]:
### Function to get the column names:
def get_column_names(columns_position):
    column_names = []
    for col in columns_position:
        column_names.append(col[0])
    return column_names

In [75]:
column_names = get_column_names(columns_positions)
print(column_names)

[' Booking Date ', ' Txn Date ', ' Booking Text ', ' Value Date ', ' Debit ', ' Credit ', ' Balance ']


## Writing a function to iterate through every line to get the table values populated in a pandas dataframe and returned.
1. Iterating through every unique value of y_min from the list - y_positions
2. For every y_position - i.e. a line, I am parsing the soup to get all the <line> tag details.
3. Now for every y-position, I am iterating through every line tag to get the text, xmin, xmax, ymin and ymax associated with it.
4. If there are multiple texts associated with the same <line> tag, then I am concatenating them into a single text.
5. Now using the xmin and xmax values, I am finding out which column the text belongs to:
    - I have the xmin and xmax positions for every column_name
    - for every column, I calculate the absolute error in x as (abs(xmin-xmin_col) + abs(xmax-xmax_col))
    - For whichever column the error is minimum, the text belongs to that column.
    - create a row (list) object and initialize the values to np.nan
    - Now, add the text to the corresponding index of the column that it belongs to
6. The process repeates for all the <line> object in the position given by y_positions.
7. Repeat the whole process for next value in y_positions.

### Handling end cases:
1. I am only considering those texts that have ymin values greater than the ymin value of the column_names
i.e. Only the lines that come after the column_names.
2. I am only considering the text that have more than 2 letters 
- this is because the pdftotext has also converted the watermark in certain places. 
- Since the watermark text is big, only a single character or 2 characters max seems to appear in the <line> section while parsing and we want to avoid them.
3. The column - "Booking Text" has multiple lines.
- So, If there is only 1 column in a row and that corresponds to "Booking Text" column, 
then instead of appending the entire row in the dataframe, I am just appending the text to the text of "Booking Text" column in the last row in the dataframe.

In [76]:
### Function to iterate through every line to get the table values populated in a pandas dataframe and returned.
def get_dataFrame(columns_positions, y_positions, lines, y_col_positions):
    df_rows = pd.DataFrame()
    for y_pos in y_positions.iterrows():
        line_list = []
        row = []
        for line in lines:
            if math.floor(float(line.get('ymin'))) == math.floor(float(y_pos[1]["ymin"])):
                s = line.get_text()
                s = ''.join(s)
                s = s.replace("\n", " ")
                word = s
                xmin = float(line.get('xmin'))
                xmax = float(line.get('xmax'))
                s = (s, float(line.get('xmin')), float(line.get('xmax')), float(line.get('ymin')), float(line.get('ymax')))
                error = []
                for i in range(len(columns_positions)):
                    row.append(np.nan)
                    x1 = float(columns_positions[i][1])
                    x2 = float(columns_positions[i][2])
                    error.append(abs(xmin-x1) + abs(xmax-x2))
                    
                min_index = error.index(min(error)) 
                if re.match("^ \d\d\.\d\d\.\d\d\d\d $",word):
                    word = word.replace(" ","")
                    word = datetime.strptime(word, '%d.%m.%Y')
                    word = word.strftime("%Y/%m/%d")
                    
                if ((min_index == 6) | (min_index == 5) | (min_index == 4)):
                    word = word.replace(" ","")
                    word = word.replace(",","")
                    
                row[min_index] = word
                s = (s,columns_positions[min_index][0])
                
                line_list.append(s)
        line_list = sorted(line_list, key=lambda x: x[0][1])
        if (len(line_list) > 1) & (line_list[0][0][3]>y_col_positions[0]):
            df_rows = df_rows.append([row[:len(columns_positions)]], ignore_index=True)
        
        else:
            if(line_list[0][1] == " Booking Text ") & (len(line_list[0][0][0])>4) & (line_list[0][0][3]>y_col_positions[0]):
                df_rows[2].iloc[-1] = df_rows[2].iloc[-1] + "\n" + line_list[0][0][0]    
            
    return df_rows

In [77]:
df = get_dataFrame(columns_positions, y_positions, lines, y_col_positions)
df.columns = column_names

In [78]:
df

,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
0,2018/03/31,2018/03/31,Initial Balance,NaN,NaN,NaN,0.00
1,2018/03/31,2018/03/31,Initial Balance,NaN,NaN,NaN,0.00
2,2018/04/01,2018/04/01,VALUE DATED BALANCE BROUGHT FORWARD,2018/04/01,NaN,180431640.00,180431640.00
3,2018/04/06,2018/04/06,INTEREST-FIXED TERM LOAN \n Contract No: 3001...,2018/04/06,472500.00,NaN,179959140.00
4,2018/04/06,2018/04/06,INTEREST-FIXED TERM LOAN \n Contract No: 3001...,2018/04/06,315000.00,NaN,179644140.00
5,2018/04/06,2018/04/06,FOREX SPOT \n EUR/JPY 130.7271,2018/04/06,NaN,472500.00,180116640.00
6,2018/04/06,2018/04/06,FOREX SPOT \n EUR/JPY 130.7021,2018/04/06,NaN,315000.00,180431640.00
7,2018/04/09,2018/04/09,INTEREST-FIXED TERM LOAN \n Contract No: 3001...,2018/04/09,157500.00,NaN,180274140.00
8,2018/04/09,2018/04/09,FOREX SPOT \n EUR/JPY 131.1407,2018/04/09,NaN,157500.00,180431640.00
9,2018/04/10,2018/04/10,INTEREST-FIXED TERM LOAN \n Contract No: 3001...,2018/04/10,157500.00,NaN,180274140.00


In [80]:
df.to_excel("output1.xlsx", index=False)

In [81]:
df.isnull().sum()

 Booking Date     1
 Txn Date         1
 Booking Text     0
 Value Date       3
 Debit            8
 Credit           7
 Balance          0
dtype: int64